# Set up


## Imports

In [1]:
%matplotlib inline

/home/gus/.anaconda/envs/biorepo/lib/python3.5/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
import sys
import os
from pathlib import Path
import math

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context(context="talk", font_scale=1, rc=None)
sns.set_style('whitegrid')

In [4]:
# Load the "autoreload" extension
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
from pandas import IndexSlice as idx
import numpy as np

from collections import OrderedDict

from munch import Munch
from tqdm import tqdm

In [6]:
import engarde.decorators as ed
import pyparsing as p

In [7]:
# import crunchers.sklearn_helpers.exploration as expl
# import crunchers.statsmodels_helpers.lazy_stats as stats
# import crunchers.pandas_helpers.transformations as xform

In [8]:
import biorep_etl.data.hreg_load_recode as loading
import biorep_etl.data.field_definitions.hreg_redcap_dump as rcd

## Constants

### Get info for converting/moving this file to `reports`

In [9]:
# %%javascript
# var kernel = IPython.notebook.kernel;
# var thename = window.document.getElementById("notebook_name").innerHTML;
# var command = "IPYNB_NAME = " + "'"+thename+"'";
# kernel.execute(command);

In [10]:
# rv = Munch()

# rv.rep_id = 'yyy-mm-dd'
# rv.desc = 'blank'
# rv.ipnb = Path('{IPYNB_NAME}.ipynb'.format(IPYNB_NAME=IPYNB_NAME))
# rv.html = '{ipnb}.html'.format(ipnb=rv.ipnb.stem)
# rv.files = '{ipnb}_files'.format(ipnb=rv.ipnb.stem)
# rv.reprt_dir = '../../reports/{IPYNB_NAME}/'.format(IPYNB_NAME=IPYNB_NAME)

### Other Constants

## Paths

In [11]:
data_ = "../data/raw/registry/HarvardInflammatoryB_DATA_2017-01-20_0924.csv"
data_dict_ = "../data/raw/registry/HarvardInflammatoryBowelDiseas_DataDictionary_2017-01-20.csv"

## Functions

## Loading

In [12]:
di = loading.load_data_dict(data_dict_=data_dict_).dropna(axis=1, how='all')
di.head(2)

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
subid,registration_visit,NaN,text,Subject ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitdate,registration_visit,Form RV: Registration Visit,text,Date of visit at which patient was consented:,NaN,NaN,date_mdy,NaN,NaN,NaN,NaN,y,NaN,NaN


In [13]:
di_ = di.reset_index()

In [14]:
di_.iloc[:,0].apply(lambda x: True if 'arm' in x else False).sum()

0

In [15]:
def clean_branching_logic(ser):
    ser.str

In [16]:
ser = di['Branching Logic (Show field only if...)']

In [17]:
rules = r"""
"""

In [18]:
blogic_unique = list(ser.dropna().str.replace('"',"'").unique())

# Parser grammer for branching logic

In [19]:
x = "([ibdtype] = '0' or [newidbdiag] = '0') and [ibdcondl(6)] = '1'"
y = "([baseline_visit_arm_1][enviquest] = '1' and [visitcategory] = '1') or [enviquest2] = '1'"
z= "[meddraae25] = '99' or [meddraae24] = '99' or [meddraae23] = '99' or [meddraae22] = '99' or [meddraae21] = '99' or [meddraae20] = '99' or [meddraae19] = '99' or [meddraae18] = '99' or [meddraae17] = '99' or [meddraae166] = '99' or [meddraae15] = '99' or [meddraae14] = '99' or [meddraae13] = '99' or [meddraae12] = '99' or [meddraae11] = '99' or [meddraae10] = '99' or [meddraae9] = '99' or [meddraae8] = '99' or [meddraae7] = '99' or [meddraae6] = '99' or [meddraae5] = '99' or [meddraae4] = '99' or [meddraae3] = '99' or [meddraae2] = '99' or [meddraae1] = '99' or [meddraae26] = '99' or [meddraae27] = '99' or [meddraae28] = '99' or [meddraae29] = '99' or [meddraae16] = '99' "

In [21]:
p.alphanums

'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789'

In [22]:
# misc
lbrack = p.Literal("[").suppress()
rbrack = p.Literal("]").suppress()
lparen = p.Literal("(").suppress()
rparen = p.Literal(")").suppress()
anyquote = p.oneOf("""" '""").suppress()
empty_quotes = p.Literal("''") | p.Literal('""')

# variables
checkbox_val = p.Optional('('+p.Word(p.nums)+')')
variable_simple = p.Combine(lbrack+p.Word(p.alphanums + "_")+checkbox_val+rbrack)
variable = p.Group(p.OneOrMore(variable_simple))

# operators
operator = p.oneOf("<> >= <= > < =")

# logic gates
AND = p.Literal("and")
OR = p.Literal("or")
logic_gate = AND | OR

# comparison vals
comparison_val = (anyquote + p.Word(p.alphanums) + anyquote) | empty_quotes

atom = variable | comparison_val


def branch_parser():

    
    expr = p.operatorPrecedence( atom,
                                [(operator, 2, p.opAssoc.LEFT),
                                 (logic_gate, 2, p.opAssoc.RIGHT),]
                                )
    return expr

In [23]:
bparser = branch_parser()

In [28]:
pstring = y

print(pstring)

xparser = bparser
# list(xparser.scanString(pstring))
out = xparser.parseString(pstring)
print(out)

([baseline_visit_arm_1][enviquest] = '1' and [visitcategory] = '1') or [enviquest2] = '1'
[[[[['baseline_visit_arm_1', 'enviquest'], '=', '1'], 'and', [['visitcategory'], '=', '1']], 'or', [['enviquest2'], '=', '1']]]


In [32]:
for s in out[0][0]:
    print(s)

[['baseline_visit_arm_1', 'enviquest'], '=', '1']
and
[['visitcategory'], '=', '1']


In [332]:
parsed_logics = []

In [333]:
for l in blogic_unique:
    parsed_logics.append(bparser.parseString(l))

In [334]:
len(parsed_logics)

189

([([(['enviquest2'], {}), '=', '1'], {}), 'and', ([(['visitcategory'], {}), '=', '2'], {})], {})

In [344]:
blogic_unique

["[consent] = '0'",
 "[noconsent] = '4'",
 "[otherstudies] = '1'",
 "[prior_protocol_number(5)] = '1' or [prior_protocol_number(1)] = '1' or [prior_protocol_number(2)] = '1' or [prior_protocol_number(3)] = '1' or [prior_protocol_number(4)] = '1'",
 "[sampletype(8)] = '1'",
 "[registration_visit_arm_1][ibddiag] = '1' and [registration_visit_arm_1][consent] = '1'",
 "[registration_visit_arm_1][ibddiag] = '1'",
 "[visitcategory] = '1' or [visitcategory] = '2'",
 "[visitcategory]='1'",
 "[visitcategory]='2'",
 "[visitcategory]='2' and [ibddiagchange]='1'",
 "[ancestrynew]='1'",
 "[jeworigin]='1' or [jeworiginnew]='1'",
 "[visitcategory] = '1' or [famhisnew] = '1'",
 "([visitcategory] = '1' or [famhisnew] = '1') and [anyfamauto]='1'",
 "([visitcategory] = '1' or [famhisnew]='1') and [anyfamauto]='1'",
 "[famcd(8)] = '1'",
 "[famuc(8)] = '1'",
 "[famun(8)] = '1'",
 "([visitcategory] = '1' or [famhisnew]='1')",
 "([visitcategory] = '1' or [famhisnew]='1') and [famauto]='1'",
 "[famautotype(7)

# Convert to HTML report and move to reports folder

In [ ]:
# rv.reprt_dir

In [ ]:
# !jupyter nbconvert --to html_toc $rv.ipnb

In [ ]:
# !mkdir -p $rv.reprt_dir
## !mv -f $rv.html $rv.files $rv.reprt_dir